# MAST30034 Project 1
## Statistical Modelling

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols, glm

In [3]:
from functools import reduce 
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/16 14:09:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/16 14:09:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/16 14:09:57 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## Neutral Network

In [4]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

In [17]:
COLS = ['tip_amount', 'fare_amount', 'trip_distance_km', 'trip_time_min', 'pickup_hour_time', 'temp']

df = pd.read_parquet("../data/curated/combined")
df = df[COLS]
df

,tip_amount,fare_amount,trip_distance_km,trip_time_min,pickup_hour_time,temp
0,0.00,12.0,4.699,14.267,0,100.5
1,8.50,35.5,19.312,24.650,0,100.5
2,3.06,14.0,6.325,14.683,0,100.5
3,0.00,6.0,2.237,4.917,0,100.5
4,0.00,23.0,11.475,20.383,0,100.5
...,...,...,...,...,...,...
3435331,7.19,40.5,23.512,25.533,23,72.5
3435332,5.00,4.5,1.497,3.117,23,72.5
3435333,3.50,22.0,10.380,20.700,23,72.5
3435334,5.04,15.5,6.067,19.017,23,72.5


In [19]:
df = pd.get_dummies(df, columns=['pickup_hour_time'])
df

,tip_amount,fare_amount,trip_distance_km,trip_time_min,temp,pickup_hour_time_0,pickup_hour_time_1,pickup_hour_time_2,pickup_hour_time_3,pickup_hour_time_4,...,pickup_hour_time_14,pickup_hour_time_15,pickup_hour_time_16,pickup_hour_time_17,pickup_hour_time_18,pickup_hour_time_19,pickup_hour_time_20,pickup_hour_time_21,pickup_hour_time_22,pickup_hour_time_23
0,0.00,12.0,4.699,14.267,100.5,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8.50,35.5,19.312,24.650,100.5,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3.06,14.0,6.325,14.683,100.5,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.00,6.0,2.237,4.917,100.5,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.00,23.0,11.475,20.383,100.5,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3435331,7.19,40.5,23.512,25.533,72.5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3435332,5.00,4.5,1.497,3.117,72.5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3435333,3.50,22.0,10.380,20.700,72.5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3435334,5.04,15.5,6.067,19.017,72.5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [20]:
TARGET_COLS = ['tip_amount']

train, test = train_test_split(df, train_size=0.8, random_state=0)

X_train, y_train = train.drop(TARGET_COLS, axis=1), train[TARGET_COLS]
X_test, y_test = test.drop(TARGET_COLS, axis=1), test[TARGET_COLS]
'''
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)
'''

print(f'{len(X_train)} training instances, {len(X_test)} test instances')

2748268 training instances, 687068 test instances


In [21]:
from tensorflow import keras
from tensorflow.keras.layers import Dense, Normalization


In [22]:
# Setup a normalization layer and adapt it to the training set so that it knows
# what mean and sd to use when normalising
norm_layer = Normalization()
norm_layer.adapt(X_train)

In [23]:
model = keras.Sequential(
    [   
        norm_layer,                   # our normalisation layer recieves the input
        Dense(5, activation='relu'),  # the hidden layer gets the normalised result
        # Dense(3, activation='relu'),  # (in case you want to try an extra hidden layer)
        Dense(1, activation='relu')   # and the output layer has a single node which will estimate total_amount
    ]
)

In [24]:
model.compile(
    optimizer='adam',  # Adam optimises using gradient descent, is generally fast and a good choice in many cases
    loss='MSE'  # Mean Squared Error makes sense for this problem, 
                # though we could use Mean Absolute Error, or many other choices.
                # Classification outputs would use a different loss (eg. BinaryCrossentropy)
)

In [25]:
history = model.fit(
    x=X_train,
    y=y_train,
    batch_size=500,
    validation_split=0.25,
    epochs=10
)

Epoch 1/10
4123/4123 [==============================] - 6s 1ms/step - loss: 4.4851 - val_loss: 4.2919
Epoch 2/10
4123/4123 [==============================] - 3s 817us/step - loss: 4.1646 - val_loss: 4.2497
Epoch 3/10
4123/4123 [==============================] - 4s 947us/step - loss: 4.1344 - val_loss: 4.2350
Epoch 4/10
4123/4123 [==============================] - 6s 1ms/step - loss: 4.1257 - val_loss: 4.2290
Epoch 5/10
4123/4123 [==============================] - 4s 937us/step - loss: 4.1158 - val_loss: 4.2165
Epoch 6/10
4123/4123 [==============================] - 5s 1ms/step - loss: 4.1030 - val_loss: 4.2072
Epoch 7/10
4123/4123 [==============================] - 4s 942us/step - loss: 4.0909 - val_loss: 4.1962
Epoch 8/10
4123/4123 [==============================] - 2s 410us/step - loss: 4.0738 - val_loss: 4.1704
Epoch 9/10
4123/4123 [==============================] - 2s 409us/step - loss: 4.0556 - val_loss: 4.1663
Epoch 10/10
4123/4123 [==============================] - 2s 398us/step

In [29]:
comparison = y_test.copy()
comparison.loc[:, 'prediction'] = model.predict(X_test)
comparison

21471/21471 [==============================] - 8s 387us/step


,tip_amount,prediction
1179931,3.54,2.093078
829879,0.00,2.208571
3221010,1.96,1.022658
1657794,2.94,1.740582
167786,1.26,1.451589
...,...,...
2295925,3.95,2.469332
1677367,1.00,2.545162
2413680,1.89,1.270366
3069009,2.36,1.570182


In [28]:
model.evaluate(
    x=X_test,
    y=y_test,
    batch_size=100,
)

6871/6871 [==============================] - 7s 948us/step - loss: 4.1515


4.15150260925293

In [31]:
predictions = model.predict(X_test)
errors = np.array(predictions - y_test)
squared_errors = errors**2
tot_sum_squares = (np.array(y_test - y_test.mean())**2).sum()
r2 = 1 - (squared_errors.sum() / tot_sum_squares)
print(f'Model R^2: {r2:.4f}')

21471/21471 [==============================] - 7s 324us/step
Model R^2: 0.1054


## Logistic regression

In [1]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import RFormula

#eatures = 'features'
#input_cols = ['fare_amount', 'passenger_count', 'pickup_location', 'trip_distance_km', 'temp', 'dew_point', 'pressure', 'wind_speed', 'wind_direction']

formula=RFormula(formula = "tip_amount ~ fare_amount + passenger_count + pickup_location + trip_distance_km + temp + dew_point + pressure + wind_speed + wind_direction", featuresCol= "features", labelCol= "label")
output = formula.fit(sdf).transform(sdf)
model_sdf = output.select("label","features")

model_sdf = model_sdf.withColumn("label", when(model_sdf["label"] > 0, 1).otherwise(model_sdf["label"]))

# Split the data into train and test
splits = model_sdf.randomSplit([0.8, 0.2], 1234)
train = splits[0]
test = splits[1]

# Fit the model
lrModel = LogisticRegression().fit(train)
lrModel.summary
# Print the coefficients and intercept for logistic regression
# print("Coefficients: " + str(lrModel.coefficients))
# print("Intercept: " + str(lrModel.intercept))

AssertionError: 

In [ ]:
fit_weather = ols(
    formula="tip_amount ~ temp + dew_point + pressure + wind_speed + wind_direction",
    data=df
).fit()

fit_taxi = ols(
    formula="tip_amount ~ pickup_location + passenger_count + fare_amount + trip_distance_km",
    data=df
).fit()

fit_all = ols(
    formula="tip_amount ~ temp + dew_point + pressure + wind_speed + wind_direction + pickup_location + passenger_count + fare_amount + trip_distance_km",
    data=df
).fit()


## Multilayer Perceptron

In [180]:
#sdf[['tip_amount']] <= 10)
(sdf.select('tip_amount').where(sdf.tip_amount == 0).count() / sdf.select('tip_amount').count())*100


34.220367188411906

As over 34% of people dont give a tip, for simplicity, the percentron will be trained to classify an instance as either 0 (didnt tip) or 1 (tipped)

In [7]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import RFormula
sdf = spark.read.parquet("../../Project 1/DataFrames/stats_modelling")
#formula=RFormula(formula = "tip_amount ~ fare_amount + passenger_count + pickup_location + trip_distance_km + temp + dew_point + pressure + wind_speed + wind_direction", featuresCol= "features", labelCol= "label")
formula=RFormula(formula = "tip_amount ~ fare_amount + trip_distance_km + temp + trip_time_min", featuresCol= "features", labelCol= "label")
output = formula.fit(sdf).transform(sdf)
model_sdf = output.select("label","features")
model_sdf = model_sdf.withColumn("label", when(model_sdf["label"] > 0, 1).otherwise(model_sdf["label"]))
# Load training data
model_sdf = model_sdf.withColumn("label",col('label').cast("int"))

In [14]:

# Split the data into train and test
splits = model_sdf.randomSplit([0.8, 0.2], 1)
train = splits[0]
test = splits[1]

In [ ]:
layers = [4, 11, 9, 2]
trainer = MultilayerPerceptronClassifier(maxIter=10, layers=layers, seed = 1)
model = trainer.fit(train)
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print(evaluator.evaluate(predictionAndLabels))

layers = [4, 11, 9, 7, 2]
trainer = MultilayerPerceptronClassifier(maxIter=10, layers=layers, seed = 1)
model = trainer.fit(train)
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print(evaluator.evaluate(predictionAndLabels))

layers = [4, 11, 9, 7, 5, 2]
trainer = MultilayerPerceptronClassifier(maxIter=10, layers=layers, seed = 1)
model = trainer.fit(train)
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print(evaluator.evaluate(predictionAndLabels))

layers = [4, 11, 9, 7, 5, 3, 2]
trainer = MultilayerPerceptronClassifier(maxIter=10, layers=layers, seed = 1)
model = trainer.fit(train)
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print(evaluator.evaluate(predictionAndLabels))

The above shows that a smaller number of hidden layers is optimal for this neutral network

In [ ]:
outputs = []
for i in range(5, 12, 2):
    for j in range(5, 12, 2):
        layers = [4, i, j, 2]
        trainer = MultilayerPerceptronClassifier(maxIter=10, layers=layers, seed = 1)
        model = trainer.fit(train)
        result = model.transform(test)
        predictionAndLabels = result.select("prediction", "label")
        evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
        outputs.append(evaluator.evaluate(predictionAndLabels))
        print(evaluator.evaluate(predictionAndLabels))
print(outputs)

[0.6601338272229761, 0.6863059061696939, 0.6762855892135433, 0.6739921146359164, 0.6912520742346965, 0.6683100568205782, 0.6584308096387014, 0.6810819615388531, 0.6840938739359775, 0.6918410769004789, 0.6939643951524354, 0.6945868473523239, 0.6719982315376751, 0.6832823591520106, 0.6888291645033544, 0.6851671676825145]

In [ ]:
outputs = []
for i in range(5, 12, 2):
    for j in range(5, 12, 2):
        layers = [4, 13, i, j, 2]
        trainer = MultilayerPerceptronClassifier(maxIter=10, layers=layers, seed = 1)
        model = trainer.fit(train)
        result = model.transform(test)
        predictionAndLabels = result.select("prediction", "label")
        evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
        outputs.append(evaluator.evaluate(predictionAndLabels))
        print(evaluator.evaluate(predictionAndLabels))
print(outputs)

[0.6841971311934357, 0.6864775168229341, 0.6784278137239076, 0.6584308096387014, 0.6846799679466203, 0.6591521561133387, 0.6921828438793897, 0.6577356410603211, 0.680402790563741, 0.6810179711257804, 0.6584308096387014, 0.6854158576969559, 0.6876933346713147, 0.6615634312241221, 0.6798225138633776, 0.6833623471683515]

From the above outputs, it can been seen that the optimal neural network is: [4, 9, 11, 2]:

In [34]:
layers = [4, 9, 11, 2]
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, seed = 1)
model = trainer.fit(train)
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print(evaluator.evaluate(predictionAndLabels))

0.7223296000744616


- 5 fold cross validation
- think of a way to graph it